In [2]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Mounted at /content/drive


In [4]:

train_dataset=[]
graphlist=[]
#from torch.utils.data import DataLoader
#%cd /content/drive/MyDrive/GNN_DATA/ADNI/ALL_NETWORKS
%cd /content/drive/MyDrive/ALL_NETWORKS

for i in range(1, 110):
    if os.path.isfile('AD_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('AD_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp=torch.div(tmp,tmp.max())
      graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

for i in range(1, 110):
    if os.path.isfile('CN_'+'{0:0>3}'.format(i)+'.csv'):
      tmp=pd.read_csv('CN_'+'{0:0>3}'.format(i)+'.csv', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp=torch.div(tmp, tmp.max())
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/10M6M6of9sCS0_CWq5d9EF6LhBCK9klNf/ALL_NETWORKS


In [5]:
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=False)

In [6]:
class E2EBlock(torch.nn.Module):
    def __init__(self, in_planes, planes,bias=True):
        super(E2EBlock, self).__init__()
        self.d = 90
        self.cnn1 = torch.nn.Conv2d(in_planes,planes,(1,self.d),bias=bias)
        self.cnn2 = torch.nn.Conv2d(in_planes,planes,(self.d,1),bias=bias)       
    def forward(self, x):
        a = self.cnn1(x)
        b = self.cnn2(x)
        return torch.cat([a]*self.d,3)+torch.cat([b]*self.d,2)

class Brainnet(torch.nn.Module):
    def __init__(self):
        super(Brainnet, self).__init__()
        self.d = 90
        self.e2econv1 = E2EBlock(1,32)   ######### Changed input channel from 1 to 3
        self.e2econv2 = E2EBlock(32,64)
        self.E2N = torch.nn.Conv2d(64,1,(1,self.d))
        self.N2G = torch.nn.Conv2d(1,256,(self.d,1))
        self.dense1 = torch.nn.Linear(256,128)
        self.dense2 = torch.nn.Linear(128,30)
        self.aux = torch.nn.Linear(30,1)
      
    def forward(self, x):
        out = F.leaky_relu(self.e2econv1(x),negative_slope=0.33)
        out = F.leaky_relu(self.e2econv2(out),negative_slope=0.33) 
        out = F.leaky_relu(self.E2N(out),negative_slope=0.33)
        out = F.leaky_relu(self.N2G(out),negative_slope=0.33)
        out = out.view(out.size(0), -1)
        out = F.leaky_relu(self.dense1(out),negative_slope=0.33)
        out = F.leaky_relu(self.dense2(out),negative_slope=0.33)
        classes = self.aux(out) 
        return classes

In [9]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Brainnet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data, label in loader:  # Iterate in batches over the training/test dataset.
         label_classes = label.type(torch.float).cuda() 
         cur_batch_size = torch.tensor(len(data)).to(device)
         data = data.view(cur_batch_size, -1).to(device)
         data = data.view(cur_batch_size, 1, 90, 90)
         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         #print(label_classes.size(), "label")
         #print(pred.size(), "pred")
         label_classes = label_classes.view(-1)
         correct += int((pred.view(-1) == label_classes).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 20):
    model.train()

    for data, label in train_loader:  # Iterate in batches over the training dataset.
         label_classes = label.type(torch.float).cuda() 
         cur_batch_size = torch.tensor(len(data)).to(device)
         data = data.view(cur_batch_size, -1).to(device)
         data = data.view(cur_batch_size, 1, 90, 90)
         out = model(data.cuda())
         out = out.view(-1)
         label = label.view(-1).cuda()
         #print(label.size(), "label")
         loss = criterion(out, label.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6860
Epoch: 002, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.7041
Epoch: 003, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.9058
Epoch: 004, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6911
Epoch: 005, Train Acc: 0.5067, Test Acc: 0.5385, Loss: 0.6995
Epoch: 006, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.6912
Epoch: 007, Train Acc: 0.4933, Test Acc: 0.4615, Loss: 0.7115
Epoch: 008, Train Acc: 0.7400, Test Acc: 0.5385, Loss: 0.6827
Epoch: 009, Train Acc: 0.7133, Test Acc: 0.6154, Loss: 0.6597
Epoch: 010, Train Acc: 0.7867, Test Acc: 0.6346, Loss: 0.6026
Epoch: 011, Train Acc: 0.7867, Test Acc: 0.6731, Loss: 0.7936
Epoch: 012, Train Acc: 0.6800, Test Acc: 0.5769, Loss: 0.5302
Epoch: 013, Train Acc: 0.8000, Test Acc: 0.6731, Loss: 0.4726
Epoch: 014, Train Acc: 0.8200, Test Acc: 0.6538, Loss: 0.3613
Epoch: 015, Train Acc: 0.8800, Test Acc: 0.6923, Loss: 0.2391
Epoch: 016, Train Acc: 0.9067, Test Acc: 0.6923, Loss: 0.3353
Epoch: 0